<a href="https://colab.research.google.com/github/jella/7-days-of-code/blob/main/playstore_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto: Análise de Sentimentos em Avaliações de Aplicativos da Play Store

## Formulação do Problema
Este projeto tem como objetivo classificar avaliações de usuários como **positivas (1)** ou **negativas (0)** com base no conteúdo textual das reviews e na pontuação fornecida (score).

Com isso, é possível automatizar a identificação de experiências insatisfatórias com apps, contribuindo para a melhoria contínua dos produtos disponíveis na Play Store.

o dataset escolhido foi https://www.kaggle.com/datasets/prakharrathi25/google-play-store-reviews

## Configuração do ambiente

In [ ]:
# === Configuração para suprimir warnings ===
import warnings
warnings.filterwarnings("ignore")

# === Bibliotecas padrão ===
import re
import string
import pickle

# === Manipulação de dados e visualização ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

# === Scikit-learn: Feature engineering ===
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer

# === Scikit-learn: Modelos ===
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

# === Scikit-learn: Avaliação e Validação ===
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

# === Scikit-learn: Pipelines ===
from sklearn.pipeline import Pipeline

# === NLP com NLTK ===
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag


In [ ]:
# É necessário baixar esses pacotes para que o NLTK funcione corretamente.
print("Baixando pacotes do NLTK...")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

Baixando pacotes do NLTK...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Carga do Dataset

In [ ]:
url = 'https://raw.githubusercontent.com/jella/playstore-sentiment-analysis/refs/heads/main/dataset/reviews.csv'
df = pd.read_csv(url,delimiter=',')
df = df[['content', 'score']]
df

,content,score
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 2020 UP...",1
4,Dan Birthday Oct 28,1
...,...,...
12490,"I really like the planner, it helps me achieve...",5
12491,😁****😁,5
12492,Very useful apps. You must try it,5
12493,Would pay for this if there were even more add...,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12495 entries, 0 to 12494
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  12495 non-null  object
 1   score    12495 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 195.4+ KB


## Pre-Processamento

### Limpeza e tokenização do Texto

In [ ]:
# Inicializações
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
label_encoder = LabelEncoder()

# Remove apenas pontuações comuns (mantém emojis e letras)
def remover_simbolos(text):
    return re.sub(r'[!"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]', '', text)

# Detecta emojis (Unicode padrão)
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # símbolos e pictogramas
    u"\U0001F680-\U0001F6FF"  # transporte/mapas
    u"\U0001F1E0-\U0001F1FF"  # bandeiras
    "]+", flags=re.UNICODE
)

#  Classificação de Sentimento
def classificar_sentimento(score):
    return 1 if score >= 4 else 0  # 1 = positivo, 0 = negativo

#  Pré-processamento de Texto
def preprocessar_texto(text):
    if not isinstance(text, str):
        text = str(text) if text else ""

    text = text.lower()

    # Remove HTML e URLs
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http\S+|www\S+", " ", text)

    # Remove pontuações específicas
    text = remover_simbolos(text)

    # Adiciona espaços ao redor de emojis
    text = emoji_pattern.sub(lambda m: f" {m.group()} ", text)

    # Tokenização: separa palavras e emojis
    tokens = re.findall(r"\w+|[^\w\s]", text)

    # Remove stopwords e números
    tokens = [t for t in tokens if t not in stop_words and not t.isdigit()]

    # Lematização apenas de palavras alfabéticas
    tokens = [lemmatizer.lemmatize(t) if t.isalpha() else t for t in tokens]

    return " ".join(tokens)

#  Aplicação do pré-processamento
def preparar_dados(df):
    if 'score' not in df.columns or 'content' not in df.columns:
        raise ValueError("O DataFrame deve conter as colunas 'score' e 'content'.")

    # Remove dados neutros e ausentes
    df = df.dropna(subset=['content']).copy()
    df = df[df['score'] != 3]

    # Classifica sentimento
    df['sentiment'] = df['score'].apply(classificar_sentimento)

    # Processa texto
    df['content'] = df['content'].apply(preprocessar_texto)

    # Codifica o target
    df['target'] = label_encoder.fit_transform(df['sentiment'])

    return df[['content', 'target']]

# === Uso ===
df = preparar_dados(df)
df.head()

,content,target
0,cannot open app anymore,0
1,begging refund app month nobody replying,0
2,costly premium version approx indian rupee per...,0
3,used keep organized update made mess thing cud...,0
4,dan birthday oct,0


## Separação em conjunto de treino e conjunto de teste com holdout

In [ ]:
# Define variáveis de entrada (X) e saída (y)
X = df['content']
y = df['target']

# Divide em treino e teste (80/20), com estratificação e reprodutibilidade
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

## Vetorização

In [ ]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=1000
)

# Ajusta nos dados de treino e transforma
X_train_vec = vectorizer.fit_transform(X_train)

# Transforma os dados de teste com o mesmo vocabulário
X_test_vec = vectorizer.transform(X_test)


## Criação e avaliação de modelos: linha base

from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# === Definição dos modelos ===
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM (Linear)': LinearSVC(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier()
}

# === Avaliação e Visualização ===
scoring = ['accuracy', 'precision', 'recall', 'f1']
cv = 5  # número de folds

# Armazena os resultados
metrics_summary = {}

for name, model in models.items():
    scores = cross_validate(
        model,
        X_train_vec,
        y_train,
        cv=cv,
        scoring=scoring,
        return_train_score=False
    )

    # Média de cada métrica
    mean_scores = {metric: np.mean(scores[f'test_{metric}']) for metric in scoring}
    metrics_summary[name] = mean_scores

    print(f"\nModelo: {name}")
    for metric, value in mean_scores.items():
        print(f"  {metric.capitalize()}: {value:.4f}")

# === Visualização com Boxplot de F1-Score ===
f1_results = [
    cross_validate(model, X_train_vec, y_train, cv=cv, scoring='f1')['test_score']
    for model in models.values()
]

plt.figure(figsize=(12, 7))
plt.boxplot(f1_results, labels=models.keys())
plt.title("Comparação de F1-Score entre os Modelos")
plt.ylabel("F1-Score")
plt.grid(True, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === Tabela de resumo das métricas ===
df_metrics = pd.DataFrame(metrics_summary).T
df_metrics = df_metrics[['accuracy', 'precision', 'recall', 'f1']]  # ordem
df_metrics = df_metrics.round(4)

import ace_tools as tools; tools.display_dataframe_to_user(name="Resumo de Métricas por Modelo", dataframe=df_metrics)

## Criação e avaliação de modelos: dados padronizados e normalizados

In [ ]:
# === Modelos ===
modelos = [
    ('LR', LogisticRegression(max_iter=200)),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('NB', MultinomialNB()),
    ('SVM', LinearSVC(max_iter=10000))
]

# === Pré-processadores ===
transformacoes = {
    'original': None,
    'padronizado': StandardScaler(with_mean=False),
    'normalizado': Normalizer()
}

# === Vetorização ===
df['content'] = df['content'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
X_text = df['content']
y = df['target']

vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = vectorizer.fit_transform(X_text)

# === Avaliação ===
all_scores = []  # para gráficos
resumo_metricas = {}  # para DataFrame de comparação

for trans_nome, transformador in transformacoes.items():
    if transformador is not None:
        X_proc = transformador.fit_transform(X_tfidf.toarray())
    else:
        X_proc = X_tfidf

    for nome, modelo in modelos:
        pipeline_id = f"{nome}-{trans_nome}"
        try:
            cv_scores = cross_validate(
                modelo,
                X_proc,
                y,
                cv=kfold,
                scoring=scoring_metrics,
                return_train_score=False
            )

            # Armazenar métricas médias
            metricas = {m: np.mean(cv_scores[f'test_{m}']) for m in scoring_metrics}
            resumo_metricas[pipeline_id] = metricas

            # Para boxplot F1
            all_scores.append(cv_scores['test_f1'])
            print(f"{pipeline_id}: " + " | ".join(f"{k}: {v:.3f}" for k, v in metricas.items()))

        except Exception as e:
            print(f"Erro em {pipeline_id}: {e}")

# === Visualização (F1-score) ===
plt.figure(figsize=(18, 8))
plt.boxplot(all_scores)
plt.title('📊 Comparação de F1-Score - Modelos + Transformações')
plt.xticks(ticks=np.arange(1, len(resumo_metricas) + 1), labels=resumo_metricas.keys(), rotation=90)
plt.ylabel('F1-Score')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# === DataFrame de métricas ===
df_resultados = pd.DataFrame(resumo_metricas).T
df_resultados = df_resultados[['accuracy', 'precision', 'recall', 'f1']].round(4)

import ace_tools as tools; tools.display_dataframe_to_user(name="Métricas por Modelo/Transformação", dataframe=df_resultados)


NameError: name 'Normalizer' is not defined

OBS:

No scikit-learn, a **A normalização L2** já é aplicada por padrão no TfidfVectorizer, tornando desnecessária qualquer normalização adicional a jusante do pipeline.

**Padronização (standardization)** geralmente não oferece ganhos significativos para modelos lineares que utilizam features TF-IDF. Isso acontece porque os valores de TF-IDF já possuem uma escala relativa e consistente por sua própria natureza.

### Otimização dos hiperparâmetros

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# === Configurações ===
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = 'accuracy'

# === Definição dos modelos e hiperparâmetros ===
modelos_parametros = {
    'Logistic Regression': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=1000)),
            ('clf', LogisticRegression(max_iter=1000))
        ]),
        'params': {
            'clf__C': [0.01, 0.1, 1, 10],
            'clf__penalty': ['l2'],
            'tfidf__ngram_range': [(1, 1), (1, 2)],
            'tfidf__min_df': [1, 3],
            'tfidf__max_df': [0.9, 0.95]
        }
    },
    'MultinomialNB': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=1000)),
            ('clf', MultinomialNB())
        ]),
        'params': {
            'clf__alpha': [0.01, 0.1, 1, 10],
            'tfidf__ngram_range': [(1, 1), (1, 2)],
            'tfidf__min_df': [1, 3],
            'tfidf__max_df': [0.9, 0.95]
        }
    },
    'Linear SVC': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=1000)),
            ('clf', LinearSVC(max_iter=10000))
        ]),
        'params': {
            'clf__C': [0.01, 0.1, 1, 10],
            'tfidf__ngram_range': [(1, 1), (1, 2)],
            'tfidf__min_df': [1, 3],
            'tfidf__max_df': [0.9, 0.95]
        }
    }
}

# === Execução dos GridSearchCVs ===
melhores_resultados = {}

for nome_modelo, config in modelos_parametros.items():
    print(f"\n🔍 Otimizando {nome_modelo}...")

    grid = GridSearchCV(
        estimator=config['pipeline'],
        param_grid=config['params'],
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        verbose=1
    )

    grid.fit(X, y)

    melhores_resultados[nome_modelo] = {
        'score': grid.best_score_,
        'params': grid.best_params_
    }

    print(f"✅ Melhor score {nome_modelo}: {grid.best_score_:.4f}")
    print(f"🔧 Melhores parâmetros {nome_modelo}: {grid.best_params_}")

# === Resumo final ===
print("\n📊 Resumo Geral dos Modelos:")
for modelo, resultado in melhores_resultados.items():
    print(f"{modelo:<18} | Score: {resultado['score']:.4f} | Params: {resultado['params']}")


## Finalização do Modelo

In [ ]:
# Pipeline com TF-IDF + Naive Bayes
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Grid de hiperparâmetros para otimização
param_grid_nb = {
    'tfidf__max_df': [0.7, 0.9],
    'tfidf__min_df': [1, 3],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__alpha': [0.5, 1, 2]
}

# Busca com validação cruzada
gs_nb = GridSearchCV(pipeline_nb, param_grid_nb, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
gs_nb.fit(X_train, y_train)

# Exibe os melhores parâmetros
print("Melhor score NB:", gs_nb.best_score_)
print("Melhores params NB:", gs_nb.best_params_)

## Salvando os arquivos

In [ ]:
# Salvar modelo treinado
joblib.dump(gs_nb.best_estimator_, 'modelo_nb_otimizado.joblib')

# Salvar também o LabelEncoder (caso esteja usando)
joblib.dump(label_encoder, 'label_encoder.joblib')

## Simulando a aplicação do modelo em dados não vistos

In [ ]:
# Carrega modelo e encoder
modelo = joblib.load('modelo_nb_otimizado.joblib')
encoder = joblib.load('label_encoder.joblib')

# Função de predição
def prever_sentimento(texto):
    texto_processado = preprocessar_texto(texto)
    pred = modelo.predict([texto_processado])
    return encoder.inverse_transform(pred)[0]

# Exemplos de uso
ex1 = "😍😍😍😍😍😍😍😍😍"
ex2 = "I hate this app, it's full of bugs 😡"

print("Exemplo 1:", prever_sentimento(ex1))  # Esperado: 'positivo'
print("Exemplo 2:", prever_sentimento(ex2))  # Esperado: 'negativo'

O modelo desenvolvido demonstrou ser eficaz na tarefa de classificar avaliações em inglês da Play Store como positivas ou negativas, utilizando exclusivamente o texto fornecido pelos usuários. A abordagem envolveu etapas clássicas de pré-processamento, vetorização com TF-IDF e experimentação com diversos algoritmos de classificação supervisionada.

Entre os algoritmos avaliados, o Naive Bayes combinado com TF-IDF se destacou pelo melhor equilíbrio entre desempenho, simplicidade e eficiência computacional. Esse modelo alcançou resultados consistentes, tornando-se uma escolha sólida para aplicações como MVPs, protótipos ou sistemas embarcados, onde o custo computacional é uma restrição relevante.

Por outro lado, observou-se que abordagens clássicas apresentam limitações ao lidar com nuances linguísticas mais complexas — como ambiguidade, ironia e gírias. Nessas situações, modelos de linguagem mais modernos, como os baseados em transformers (ex: BERT, RoBERTa), tendem a oferecer resultados superiores, porém com maior demanda de recursos e complexidade de implementação.

De modo geral, o projeto evidencia que técnicas tradicionais de NLP ainda têm espaço e podem entregar valor real em cenários práticos, especialmente na automação da triagem de feedbacks de usuários, contribuindo para decisões de produto mais ágeis e embasadas.

